In [1]:
import sys
sys.path.insert(1, 'Classes/')
import pandas as pd
import numpy as np
import random
from tqdm import tqdm
from multiprocessing import current_process

from ScoreGetter import ScoreGetter
from dataset_utils import checkIfEarlyMidEnd
from dataset_utils import encodeBoard
from dataset_utils import getColumns

In [2]:
df = pd.read_csv('Datasets/raw_dataset_13M.csv')
boards = df['board'].values

#### We load an engine to get a score from the positions.

In [3]:
score_getter = ScoreGetter('/home/gaetan/Téléchargements/stockfish/stockfish', 'eval', 'go depth 1')

#### We create our dataset by getting an equal number of start, middle and end game positions and encoding them.

In [4]:
total_size = 1000000
batch_size = total_size / 3.0

earlies = []
mids = []
ends = []


current = current_process()
pos = current._identity[0]-1 if len(current._identity) > 0 else 0
pbar = tqdm(total=total_size*3, desc='Splitting and encoding', position=pos)


for i in range(13):
    print(i+1)
    data = []
    for j in range(i * 1000000, i * 1000000 + total_size):
        board = boards[j]
        data.append(np.append(encodeBoard(board), score_getter.getScore(board)))
        pbar.update(1)
    df = pd.DataFrame(data, columns=np.append(getColumns(), 'cp (Stockfish 13)'))
    df.to_csv('Datasets/dataset' + str(i+1) + '.csv', index=False)
    
pbar.close()


Splitting and encoding:   0%|          | 1/3000000 [00:00<143:07:10,  5.82it/s]

1


Splitting and encoding:  33%|███▎      | 1000089/3000000 [20:18<143:47:45,  3.86it/s]

2


Splitting and encoding:  67%|██████▋   | 2000088/3000000 [40:40<69:00:19,  4.03it/s] 

3


Splitting and encoding: 100%|██████████| 3000000/3000000 [1:01:04<00:00, 818.66it/s]


'\nfor i in range(4000000, len(boards)):\n    board = boards[i]\n    part = checkIfEarlyMidEnd(board)\n    \n    if len(earlies) < batch_size and part == "early_game":\n        earlies.append(np.append(encodeBoard(board), score_getter.getScore(board)))\n        pbar.update(1)\n    \n    elif len(mids) < batch_size and part == "mid_game":\n        mids.append(np.append(encodeBoard(board), score_getter.getScore(board)))\n        pbar.update(1)\n        \n    elif len(ends) < batch_size and part == "end_game":\n        ends.append(np.append(encodeBoard(board), score_getter.getScore(board)))\n        pbar.update(1)\n    \n    if len(earlies) >= batch_size and len(mids) >= batch_size and len(ends) >= batch_size:\n        break\n    \npbar.close()\n'